## Import Library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

## Import Data

In [2]:
train_df = pd.read_csv('data_train.csv')
x_train = train_df.drop(columns=['CropEncode'])
y_train = train_df['CropEncode']
x_train.head()

,Nitrogen,Phosphorus,Potassium,Temperature,Humidity,pH_Value,Rainfall
0,17,16,14,16.396243,92.181519,6.625539,102.944161
1,37,79,19,27.543848,69.347863,7.143943,69.408782
2,7,73,25,27.521856,63.132153,7.288057,45.208411
3,101,70,48,25.360592,75.031933,6.012697,116.553145
4,0,17,30,35.474783,47.972305,6.279134,97.790725


In [3]:
test_df = pd.read_csv('data_test.csv')
x_test = test_df.drop(columns=['CropEncode'])
y_test = test_df['CropEncode']
x_test.head()

,Nitrogen,Phosphorus,Potassium,Temperature,Humidity,pH_Value,Rainfall
0,101,17,47,29.494014,94.729813,6.185053,26.308209
1,98,8,51,26.179346,86.522581,6.259336,49.430510
2,59,62,49,43.360515,93.351916,6.941497,114.778071
3,44,60,55,34.280461,90.555616,6.825371,98.540477
4,30,137,200,22.914300,90.704756,5.603413,118.604465


## SVM

In [4]:
%pip install tensorflow distutils

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement distutils (from versions: none)
ERROR: No matching distribution found for distutils


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [6]:
ann_model = Sequential()
ann_model.add(Dense(32, input_dim=7, activation='relu'))
ann_model.add(Dense(16, activation='relu'))
ann_model.add(Dense(1))

In [7]:
ann_model.compile(optimizer='adam', loss='mean_squared_error')

In [8]:
ann_model.fit(x_train, y_train, epochs=100, batch_size=10)

Epoch 1/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 534us/step - loss: 137.8309 
Epoch 2/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 512us/step - loss: 19.8728
Epoch 3/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 515us/step - loss: 16.9446
Epoch 4/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step - loss: 14.8388
Epoch 5/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 505us/step - loss: 14.3618
Epoch 6/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 518us/step - loss: 12.2475
Epoch 7/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 526us/step - loss: 12.8353
Epoch 8/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 499us/step - loss: 11.1009
Epoch 9/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 531us/step - loss: 11.3011
Epoch 10/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 517us/step - loss: 11.8469
Epoch 11/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 519us/step - loss: 10.3695
Epoch 12/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 509us/step - loss: 11.2656
Epoch 13/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 509us/step - loss: 10.6475
Epoch 14/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 508us/step - loss: 9.1525


In [9]:
from sklearn.metrics import mean_squared_error
from math import sqrt

train_pred = ann_model.predict(x_train)
mse = mean_squared_error(y_train, train_pred)
rmse = sqrt(mse)
print('Train RMSE:', rmse)

55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 550us/step
Train RMSE: 2.06779171855936


In [10]:
test_pred = ann_model.predict(x_test)
mse = mean_squared_error(y_test, test_pred)
rmse = sqrt(mse)
print('Test RMSE:', rmse)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Test RMSE: 2.370917388091793


## Keras-Tuner

In [12]:
%pip install keras-tuner

   ---------------------------------------- 0.0/129.1 kB ? eta -:--:--
   --------- ------------------------------ 30.7/129.1 kB 1.4 MB/s eta 0:00:01
   ------------ -------------------------- 41.0/129.1 kB 653.6 kB/s eta 0:00:01
   ---------------------------- ----------- 92.2/129.1 kB 1.3 MB/s eta 0:00:01
   -------------------------------------- 129.1/129.1 kB 951.3 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [13]:
from kerastuner import HyperModel, RandomSearch

In [17]:
class RegressionHyperModel(HyperModel):
    def build(self, hp):
        model = Sequential()
        model.add(Dense(
            units=hp.Int('units_input', min_value=32, max_value=128, step=16),
            activation='relu',
            input_dim=x_train.shape[1]
        ))
        for i in range(hp.Int('num_layers', 1, 3)):
            model.add(Dense(
                units=hp.Int(f'units_{i}', min_value=32, max_value=128, step=16),
                activation='relu'
            ))
        model.add(Dense(1))  # Output layer untuk regresi
        model.compile(
            optimizer=hp.Choice('optimizer', ['adam', 'rmsprop']),
            loss='mean_squared_error'
        )
        return model

In [18]:
hypermodel = RegressionHyperModel()

In [19]:
tuner = RandomSearch(
    hypermodel,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=2,
    directory='hyperparam_tuning',
    project_name='regression_model'
)

In [20]:
tuner.search(x_train, y_train, epochs=50, validation_split=0.2, batch_size=10)

Trial 10 Complete [00h 00m 17s]
val_loss: 2.8108410835266113

Best val_loss So Far: 2.1096922159194946
Total elapsed time: 00h 02m 44s


In [23]:
best = tuner.get_best_hyperparameters(num_trials=1)[0]

In [25]:
ann_model_tuning = tuner.hypermodel.build(best)
ann_model_tuning.fit(x_train, y_train, epochs=100, batch_size=10)

Epoch 1/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 556us/step - loss: 34.3851 
Epoch 2/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 567us/step - loss: 14.1593
Epoch 3/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 549us/step - loss: 11.5010
Epoch 4/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 556us/step - loss: 9.4324
Epoch 5/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 564us/step - loss: 7.4482
Epoch 6/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 583us/step - loss: 6.5057
Epoch 7/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 556us/step - loss: 6.3339
Epoch 8/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 572us/step - loss: 5.9015
Epoch 9/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 574us/step - loss: 5.9684
Epoch 10/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 559us/step - loss: 5.3467
Epoch 11/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 579us/step - loss: 5.0939
Epoch 12/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 565us/step - loss: 4.3225
Epoch 13/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 563us/step - loss: 4.6445
Epoch 14/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 556us/step - loss: 4.9654
Epoch 15/10

In [26]:
train_pred_tuning = ann_model_tuning.predict(x_train)
mse = mean_squared_error(y_train, train_pred_tuning)
rmse = sqrt(mse)
print('Train RMSE:', rmse)

55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 565us/step
Train RMSE: 1.409521426006668


In [27]:
test_pred_tuning = ann_model_tuning.predict(x_test)
mse = mean_squared_error(y_test, test_pred_tuning)
rmse = sqrt(mse)
print('Test RMSE:', rmse)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Test RMSE: 1.7999136400083295


## Hasil

Skor RMSE sebelum dan sudah dilakukan tuning:
- ANN (sebelum tuning):
  - Train = 2.06779171855936
  - Test = 2.370917388091793
- ANN (sesudah tuning):
  - Train = 1.409521426006668
  - Test = 1.7999136400083295

Kesimpulan:
- Penurunan skor RMSE pada set pengujian menunjukkan bahwa model yang dituning memiliki kemampuan generalisasi yang lebih baik, yaitu kemampuan untuk memprediksi dengan akurasi yang lebih tinggi pada data yang tidak dilihat selama training.
- Proses tuning terbukti efektif dalam meningkatkan kinerja model secara keseluruhan. Dengan mencoba berbagai konfigurasi hyperparameter, kita dapat menemukan kombinasi yang mengurangi kesalahan prediksi baik pada data train maupun test.